In [10]:
import pandas as pd
import numpy as np
import string, re
import swifter
import pickle 

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk import bigrams 
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer

from nltk.corpus import stopwords
# import nltk
# nltk.download('stopwords')

import spacy
nlp = spacy.load("en_core_web_sm")
from sklearn.decomposition import NMF, TruncatedSVD
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity

file_path = 'SriLankanRecipes.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)
#df.head()
df = df.set_index('ID')
df = df.dropna(how='any')    #1292

In [12]:
import pandas as pd
import re

# List of measurement words
measure_words = [
    'Servings', 'Ingredients' ,'INGREDIENTS' ,'For' ,'the' , 'filling' ,
    'All' , 'purpose','you want' ,'you need','our','sharp','fork', 
    'half','Filling' , 'finely' , 'into' , 'and' , 'for' ,'with',
    'heat', 'inch' ,'piece','asian','per', 'taste','sprig','deep frying',
    'bottle', 'bottles', 'box', 'boxes', 'bunch', 'bunches', 'bushel', 'bushels',
    'can', 'cans', 'container', 'cup', 'cups', 'carton', 'cartons', 'dash', 'dashes',
    'drop', 'drops', 'fl', 'fl.', 'fluid', 'jar', 'jars', 'ounce', 'ounces', 
    'gallon', 'gallons', 'glass', 'glasses', 'gram', 'grams', 'kg', 'kgs', 'lb', 'lbs',
    'liter', 'liters',  'large', 'medium', 'ml', 'mls', 'package', 'pkg', 'small',
    'to taste', 'pinch', 'pinches', 'pint', 'pints', 'pound', 'pounds', 'qt', 'qts',
    'quart', 'quarts', 'scoop', 'scoops', 'sliced', 'slivered', 'stick', 'sticks',
    'tablespoon', 'tablespoons', 'tbs', 'tbsp', 'tbsps', 'teaspoon', 'teaspoons', 'tsp',
    'tsps', 'whole' , 'shredded', 'boiled', 'mashed ', 'Roasted', 'Ripe',
    'unroasted', 'green' , 'powder', 'chopped', 'mashed ', 'Steamed', 'Sri Lankan'
]

# Function to clean the ingredients text
def clean_once(text):
    if not isinstance(text, str):  # Check if text is a string
        return []
    
    ingredlist = []
    for ingred in text.split("#item,"):
        ingred = ingred.replace('#item', '')  # Scraping artifact on last item
        ingred = re.sub(r'\([^)]*\)', '', ingred)  # Remove anything inside parentheses
        ingred = ingred.split(',')[0]  # Remove anything after a comma
        ingred = re.sub(r'\w*\d\w*', ' ', ingred)  # Remove numeric characters
        ingred = ingred.replace('⁄', ' ')  # Fraction slashes are annoying
        ingred = ' ' + ingred + ' '  # Padding in case measure_word is first or last
        
        for measure_word in measure_words:  # Remove measure words by themselves
            ingred = ingred.replace(measure_word, ' ')
        
        ingred = ingred.strip()
        # Remove words with one or two letters
        ingred = ' '.join([word for word in ingred.split() if len(word) > 2])
        
        # Remove non-alphabetic characters
        ingred = re.sub(r'[^a-zA-Z\s]', '', ingred)
        
        if ingred and not ingred.isupper():  # Important for multi-part recipes
            ingredlist.append(ingred)
    
    return ingredlist


# Applying the cleaning function to the DataFrame
df['CleanIngredients'] = df['Ingredients'].apply(clean_once)

# Drop rows where 'Ingredients' or 'CleanIngredients' columns contain NaN values
df = df.dropna(subset=['Ingredients', 'CleanIngredients'])


# Print the cleaned ingredients column
print(df['CleanIngredients'])

ID
1      [chicken potatoes cheddar cheese garlic onion ...
2      [chicken potatoes cheddar cheese habanero chil...
3      [bone chicken curry red chili onion cloves gar...
4      [beans chili cinnamon pearl onions Asian shall...
5      [cooked rice eggs gheecooking oil onion ginger...
                             ...                        
382    [ Payatham Urundai Mung bean Green moong dhal ...
383    [Butter Granulated sugar Egg Vanilla extract B...
384    [peanut Butter Brown sugar White sugar Vanilla...
385        [palmyra fruit juice Sugar Water Cooking oil]
386    [Banana Wood apple Papaya Mango Raisins cashew...
Name: CleanIngredients, Length: 379, dtype: object


In [13]:
df

Title  \
ID                                                       
1    Chicken cheese bites with Habanero (Kochchi Bi...   
2        Crispy Chicken rolls with Habanero and cheese   
3                         Sri Lankan Red Chicken Curry   
4             Vegan green bean curry with coconut milk   
5                             Egg Biryani (Stir-fried)   
..                                                 ...   
382  HOW TO MAKE PAYATHAM PANIYARAM URUNDAI WITH JA...   
383  EASY VANILLA BUTTER CAKE RECIPE WITHOUT MILK H...   
384   EASY PEANUT COOKIES WITHOUT EGGS & PEANUT BUTTER   
385  HOW TO MAKE PANANGAI PANIYARAM | PALMYRA FRUIT...   
386    SIMPLE & FRESH FRUIT SALAD RECIPE WITHOUT HONEY   

                                           Ingredients  \
ID                                                       
1    Ingredients\nServings\nFor the filling\n1 cup ...   
2    Ingredients\nServings\nFor the Filling\n2 cup ...   
3    Ingredients\nServings\n750 g bone in chicken p...   
4    Ingredients\nServings\n2 cup sliced green bean...   
5    Ingredients\nServings\n2 cup cooked rice\n2 bo...   
..                                                 ...   
382  INGREDIENTS\n-------For Payatham Urundai------...   
383  INGREDIENTS\nButter - 500g\nAll purpose flour ...   
384  INGREDIENTS\nRoasted peanut - 200g\nAll purpos...   
385  INGREDIENTS\nRipe palmyra fruit juice - 1000ml...   
386  INGREDIENTS\nBanana - 300g\nWood apple - 500g\...   

                                           Instruction  \
ID                                                       
1    Making filling and the balls\nMix everything e...   
2    Instructions\nMake the filling\n\nGrind the ha...   
3    Grind cardamom and cloves using a mortar and p...   
4    Instructions\nAdd everything together except t...   
5    Instructions\nHeat ghee/oil in a pan. Add Cumi...   
..                                                 ...   
382  INSTRUCTIONS\nFirstly, dry roast the mung bean...   
383  INSTRUCTIONS\nGet ready with butter, granulate...   
384  INSTRUCTIONS\nTake the large bowl and add the ...   
385  INSTRUCTIONS\nTake the ripe palmyra fruits.\nR...   
386  INSTRUCTIONS\nTake the banana, wood apple, pap...   

                                      CleanIngredients  \
ID                                                       
1    [chicken potatoes cheddar cheese garlic onion ...   
2    [chicken potatoes cheddar cheese habanero chil...   
3    [bone chicken curry red chili onion cloves gar...   
4    [beans chili cinnamon pearl onions Asian shall...   
5    [cooked rice eggs gheecooking oil onion ginger...   
..                                                 ...   
382  [ Payatham Urundai Mung bean Green moong dhal ...   
383  [Butter Granulated sugar Egg Vanilla extract B...   
384  [peanut Butter Brown sugar White sugar Vanilla...   
385      [palmyra fruit juice Sugar Water Cooking oil]   
386  [Banana Wood apple Papaya Mango Raisins cashew...   

                                          Ingredients1  
ID                                                      
1    1 cup finely shredded chicken\n3â„4 cup boile...  
2    2 cup finely shredded chicken\n1 1â„2 cup boi...  
3    750 g bone in chicken pieces\n1 tbsp unroasted...  
4    2 cup sliced green beans\n1 green chili (optio...  
5    2 cup cooked rice\n2 boiled eggs\n1 tbsp ghee/...  
..                                                 ...  
382  INGREDIENTS\n-------For Payatham Urundai------...  
383  INGREDIENTS\nButter - 500g\nAll purpose flour ...  
384  INGREDIENTS\nRoasted peanut - 200g\nAll purpos...  
385  INGREDIENTS\nRipe palmyra fruit juice - 1000ml...  
386  INGREDIENTS\nBanana - 300g\nWood apple - 500g\...  

[379 rows x 5 columns]

In [12]:
import pandas as pd

# List of measurement words
words = [
    'INSTRUCTIONS', 'Instructions','Method', 'Process'
]

def clean_instructions(text):
    if not isinstance(text, str):  # Check if text is a string
        return []
    
    instructionslist = []
    for ingred in text.split("#item,"):
        ingred = ingred.replace('\n', ' ')    
        ingred = ingred.replace('#item', '')  # Scraping artifact on last item
        ingred = ' ' + ingred + ' '  # Padding in case measure_word is first or last
        
        for word in words:  # Remove measure words by themselves
            ingred = ingred.replace(word, ' ')
        
        ingred = ingred.strip()  # Remove leading and trailing spaces
        
        if ingred and not ingred.isupper():  # Important for multi-part recipes
            instructionslist.append(ingred)
    
    return instructionslist



# Applying the cleaning function to the DataFrame
df['CleanInstructions'] = df['Instruction'].apply(clean_instructions)

# Drop rows where 'Instruction' or 'CleanInstructions' columns contain NaN values
df = df.dropna(subset=['Instruction', 'CleanInstructions'])

# Print the cleaned instructions column
print(df['CleanInstructions'])


ID
1      [Making filling and the balls Mix everything e...
2      [Make the filling  Grind the habaneros and gre...
3      [Grind cardamom and cloves using a mortar and ...
4      [Add everything together except thick coconut ...
5      [Heat ghee/oil in a pan. Add Cumin seeds, cinn...
                             ...                        
382    [Firstly, dry roast the mung bean under medium...
383    [Get ready with butter, granulated sugar, all-...
384    [Take the large bowl and add the butter. Also ...
385    [Take the ripe palmyra fruits. Remove the blac...
386    [Take the banana, wood apple, papaya, and mang...
Name: CleanInstructions, Length: 379, dtype: object


In [11]:
import pandas as pd

# List of measurement words
words = [
    'Ingredients', 'INGREDIENTS','Servings', 'Process'
]

def clean_instructions(text):
    if not isinstance(text, str):  # Check if text is a string
        return []
    
    instructionslist = []
    for ingred in text.split("#item,"):
        ingred = ingred.replace('\n', "','")        
        ingred = ingred.replace('#item', '')  # Scraping artifact on last item
        ingred = ' ' + ingred + ' '  # Padding in case measure_word is first or last
        ingred = ingred.strip()  # Remove leading and trailing spaces
        
        for word in words:  # Remove measure words by themselves
            ingred = ingred.replace(word, ' ')
        

        
        if ingred and not ingred.isupper():  # Important for multi-part recipes
            instructionslist.append(ingred)
        

    
    return instructionslist



# Applying the cleaning function to the DataFrame
df['Ingredients2'] = df['Ingredients'].apply(clean_instructions)

# Drop rows where 'Instruction' or 'CleanInstructions' columns contain NaN values
df = df.dropna(subset=['Ingredients', 'Ingredients2'])

# Print the cleaned instructions column
print(df['Ingredients2'])


ID
1      [ ',' ','For the filling','1 cup finely shredd...
2      [ ',' ','For the Filling','2 cup finely shredd...
3      [ ',' ','750 g bone in chicken pieces','1 tbsp...
4      [ ',' ','2 cup sliced green beans','1 green ch...
5      [ ',' ','2 cup cooked rice','2 boiled eggs','1...
                             ...                        
382    [ ','-------For Payatham Urundai-------','Mung...
383    [ ','Butter - 500g','All purpose flour - 500g'...
384    [ ','Roasted peanut - 200g','All purpose flour...
385    [ ','Ripe palmyra fruit juice - 1000ml (3 Palm...
386    [ ','Banana - 300g','Wood apple - 500g','Papay...
Name: Ingredients2, Length: 379, dtype: object


In [33]:
df.to_csv('CleanSrilankanRecipes.csv', index=False)

In [13]:
df

Title  \
ID                                                       
1    Chicken cheese bites with Habanero (Kochchi Bi...   
2        Crispy Chicken rolls with Habanero and cheese   
3                         Sri Lankan Red Chicken Curry   
4             Vegan green bean curry with coconut milk   
5                             Egg Biryani (Stir-fried)   
..                                                 ...   
382  HOW TO MAKE PAYATHAM PANIYARAM URUNDAI WITH JA...   
383  EASY VANILLA BUTTER CAKE RECIPE WITHOUT MILK H...   
384   EASY PEANUT COOKIES WITHOUT EGGS & PEANUT BUTTER   
385  HOW TO MAKE PANANGAI PANIYARAM | PALMYRA FRUIT...   
386    SIMPLE & FRESH FRUIT SALAD RECIPE WITHOUT HONEY   

                                           Ingredients  \
ID                                                       
1    Ingredients\nServings\nFor the filling\n1 cup ...   
2    Ingredients\nServings\nFor the Filling\n2 cup ...   
3    Ingredients\nServings\n750 g bone in chicken p...   
4    Ingredients\nServings\n2 cup sliced green bean...   
5    Ingredients\nServings\n2 cup cooked rice\n2 bo...   
..                                                 ...   
382  INGREDIENTS\n-------For Payatham Urundai------...   
383  INGREDIENTS\nButter - 500g\nAll purpose flour ...   
384  INGREDIENTS\nRoasted peanut - 200g\nAll purpos...   
385  INGREDIENTS\nRipe palmyra fruit juice - 1000ml...   
386  INGREDIENTS\nBanana - 300g\nWood apple - 500g\...   

                                           Instruction  \
ID                                                       
1    Making filling and the balls\nMix everything e...   
2    Instructions\nMake the filling\n\nGrind the ha...   
3    Grind cardamom and cloves using a mortar and p...   
4    Instructions\nAdd everything together except t...   
5    Instructions\nHeat ghee/oil in a pan. Add Cumi...   
..                                                 ...   
382  INSTRUCTIONS\nFirstly, dry roast the mung bean...   
383  INSTRUCTIONS\nGet ready with butter, granulate...   
384  INSTRUCTIONS\nTake the large bowl and add the ...   
385  INSTRUCTIONS\nTake the ripe palmyra fruits.\nR...   
386  INSTRUCTIONS\nTake the banana, wood apple, pap...   

                                      CleanIngredients  \
ID                                                       
1    ['chicken and potatoes cheddar cheese garlic p...   
2    ['chicken potatoes cheddar cheese habanero gre...   
3    ['bone chicken unroasted Sri Lankan curry powd...   
4    ['green beans green chili cinnamon pearl onion...   
5    ['cooked rice eggs gheecooking oil onion ginge...   
..                                                 ...   
382  [' Payatham Urundai Mung bean Green moong dhal...   
383  ['Butter Granulated sugar Egg Vanilla extract ...   
384  ['Roasted peanut Butter Brown sugar White suga...   
385  ['Ripe palmyra fruit juice Steamed Sugar Water...   
386  ['Banana Wood apple Papaya Mango Raisins Roast...   

                                          Ingredients1  \
ID                                                       
1    1 cup finely shredded chicken\n3â„4 cup boile...   
2    2 cup finely shredded chicken\n1 1â„2 cup boi...   
3    750 g bone in chicken pieces\n1 tbsp unroasted...   
4    2 cup sliced green beans\n1 green chili (optio...   
5    2 cup cooked rice\n2 boiled eggs\n1 tbsp ghee/...   
..                                                 ...   
382  INGREDIENTS\n-------For Payatham Urundai------...   
383  INGREDIENTS\nButter - 500g\nAll purpose flour ...   
384  INGREDIENTS\nRoasted peanut - 200g\nAll purpos...   
385  INGREDIENTS\nRipe palmyra fruit juice - 1000ml...   
386  INGREDIENTS\nBanana - 300g\nWood apple - 500g\...   

                                          Ingredients2  \
ID                                                       
1    [ ',' ','For the filling','1 cup finely shredd...   
2    [ ',' ','For the Filling','2 cup finely shredd...   
3    [ ',' ','750 g bone in chicken pieces